In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 11 18:18:44 2019

@author: chris
"""

# This file contains the script to read the URLs from the SQLite database
# Start: SQLite database
# End: URLs in database

# Step2A: Read the URLs from the SQLite database
# Step2B: Scrape the URLs and write the content to the database
# Step2C: Retrieve URLs from content, write URLs to database
# Step2D: Use URLs to scrape, retrieve contents and write to database

import pandas as pd
import sqlite3
from sqlite3 import Error
import requests
import bs4
from bs4 import BeautifulSoup
from bs4.element import Comment
from lxml.html.soupparser import fromstring
import urllib.request

In [2]:
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print("conn open")
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)

    return None

In [3]:
def retrieve_data(sql, conn):
    df_data = pd.read_sql_query(sql, conn)
    return df_data

In [4]:
def scrape1(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    response_scrape1 = requests.get('http://' + url, headers=headers)
    return response_scrape1

In [5]:
def process_scrape1(resp_obj):
    soup = bs4.BeautifulSoup(resp_obj.text, 'lxml')
    text_total = soup.get_text().strip()
    return text_total

In [6]:
#def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

# def scrape2(url2):
#     hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
#     url = "https://" + url2
#     html_req = urllib.request.Request(url, headers = hdr)
#     html_resp = urllib.request.urlopen(html_req)
#     html = html_resp.read()
#     return(html)
    
# def process_scrape2(scrape2_text):
#     html_text = text_from_html(scrape2_text)
#     return(html_text)
    
# def tag_visible(element):
#     if element.parent.name in ['style', 'script', 'head', 'title', 'meta', 'document]']:
#         return False
#     if isinstance(element, Comment):
#         return False
#     return True    

# def text_from_html(body):
#     soup = BeautifulSoup(body, 'lxml')
#     texts = soup.findAll(text=True)
#     visible_texts = filter(tag_visible, texts)
#     return u" ".join(t.strip() for t in visible_texts)

In [7]:
def process_scrape(scrape2_text):
    html_text = text_from_html(scrape2_text)
    return(html_text)

In [8]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', 'document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True    

In [9]:
def text_from_html(body):
    soup = BeautifulSoup(body, 'lxml')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

In [10]:
# def write_to_db(df, conn):
#     sql2 = """INSERT INTO secondScrape VALUES
#             (
#             ?,?,?,?,?
#             )"""      
#     list_values2 = [main_company, id_link, url, scrape1_response, process1_text]       
#     cur = conn.cursor()
#     cur.execute(sql2, list_values2)
#     conn.commit()

In [11]:
def write_to_db(df, table, conn):
    df.to_sql(table, conn, if_exists="replace")    
    
# def visible_text(element):
#     if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
#         return ''
#     result = re.sub('<!--.*-->|\r|\n', '', str(element), flags=re.DOTALL)
#     result = re.sub('\s{2,}|&nbsp;', ' ', result)
#     return result

In [12]:
def main():
    database = "scrape.db"
    conn = create_connection(database)
    cols = ["Company", "ID_link", "URL", "Source_code", "Text"]
    df_all = pd.DataFrame(columns = cols)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    
    # Step 2C
    table = "firstScrape"
    sql = "SELECT * FROM " + table
    df_data = retrieve_data(sql, conn)
#     print(df_data.head())
    
    for index, row in df_data.iterrows():
        id_link = index
        main_company = row['Company']
        main_url_company = row['Website']
#         list_testURL = df_data.iloc[index][3]
        list_URL = row["Links_1st"]
        list_URL = list_URL[1:-1].split(",")

        for url in list_URL:
            if len(url) > 0:

                if main_url_company in url:
                    try:                        
#                         print("0")
                        print(url)
                        source_code = requests.get(url, headers=headers).text
                        print('2')
                        text = process_scrape(source_code)
#                         print("1")
                        df_all = df_all.append({"Company": main_company, "ID_link": id_link, "URL": url, "Source_code": source_code, "Text": text}, ignore_index=True)
#                         print("DONE!")
                    except:
                        print("no")
                        next
                        
                else:
                    try:
                        url_tot = 'https://' + main_url_company + url
#                         print("0")                
                        print(url_tot)
                        source_code = requests.get(url_tot, headers=headers).text
                        print('2')
                        text = process_scrape(source_code)
#                         print("1")
                        df_all = df_all.append({"Company": main_company, "ID_link": id_link, "URL": url, "Source_code": source_code, "Text": text}, ignore_index=True)
#                         print("DONE!")
                
                    except:
                        print("no")
                        next
    print(df_all)
    write_to_db(df_all, "secondScrape", conn)
    conn.close()
    

In [13]:
if __name__ == "__main__":
    main()

conn open
2.6.0
https://www.starlingbank.com/rss.xml
2
https://www.starlingbank.com/media/
2
https://www.starlingbank.com/current-account/overdraft/
2
https://www.starlingbank.com/online-banking/
2
https://www.starlingbank.com/banking-services/
2
https://www.starlingbank.com/business-account/overdraft/
2
https://www.starlingbank.com/send-money-abroad/send-money-to-usa/
2
https://www.starlingbank.com#
2
https://www.starlingbank.com/travel/
2
https://www.starlingbank.com/blog/business-talks/
2
https://www.starlingbank.com/docs/Starling-Bank-16-02-article-daily-mail.pdf
2
https://www.starlingbank.com/blog/turn-your-passion-into-your-profession/
2
https://www.starlingbank.com/send-money-abroad/send-money-to-thailand/
2
https://www.starlingbank.com/legal/privacy-notice/
2
https://www.starlingbank.com/blog/women-in-finance-charter-2019-update/
2
https://www.starlingbank.com/investors/
2
https://www.starlingbank.com/download/
2
https://www.starlingbank.com#scroll-section
2
https://www.starlin